In [1]:
from inference import FastGM
from inference import FastFactor
from sampling import SampleGenerator
from inference import populate_gradient_factors
from data import DataLoader
from data import create_data_loaders
from data import DataPreprocessor
from neural_networks import Trainer
from neural_networks.losses import *
import numpy as np
import torch
import torchinfo
import torch.nn as nn
from neural_networks import Net
import matplotlib.pyplot as plt

In [2]:
device = 'cuda'

In [3]:

ib = 5
# Z = 303.0859680175781
Z = -78.13997650146484
# uai_file = "/home/cohenn1/SDBE/width_under_20_problems/grid10x10.f10.uai"
# uai_file = "/home/cohenn1/SDBE/width20-30/grid20x20.f2.uai"
uai_file = "/home/cohenn1/SDBE/width20-30/pedigree18.uai"
# problem_name = "grid10x10.f10-ib" + str(ib)
# problem_name = "grid20x20.f2-ib" + str(ib)  
problem_name = "pedigree18-ib" + str(ib)  
output_path = "/home/cohenn1/SDBE/PyTorchGMs/graphs"
grid20f2_idxs = [30, 106, 213, 123, 331]
# idx = 74 # for grid 20, ib10
idx = 716
#%%
fastgm = FastGM(uai_file=uai_file, device=device)
fastgm.dope_factors()
# fastgm.get_large_message_buckets(20)

<class 'pyGMs.factor.Factor'>


In [4]:
Z+78.13997650146484

0.0

In [5]:
# populate_gradient_factors(fastgm,iB=10)
bucket=fastgm.buckets[idx]
fastgm_copy = FastGM(uai_file=uai_file, device=device)
mg, mess = fastgm.get_message_gradient(idx)
gradient_factors = fastgm.get_gradient_factors(idx)
# mg_hat = fastgm_copy.get_wmb_message_gradient(bucket_var=idx, i_bound=ib, weights='max')
mess.order_indices()
# mg_hat.order_indices()

<class 'pyGMs.factor.Factor'>


In [6]:
fastgm.config = {
    'num_samples': -1,
    'sampling_scheme': 'uniform'
}
sg = SampleGenerator(gm=fastgm, bucket=bucket, mess = mess, mg = mg)
sample_assignments = sg.sample_assignments(1000)
sample_values = sg.compute_message_values(sample_assignments)
sample_mg_values = sg.compute_gradient_values(sample_assignments, [mg])

In [7]:
data_preprocessor = DataPreprocessor(y=sample_values, mg=sample_mg_values, is_logspace=True, device = device)
dl = DataLoader(bucket=bucket, sample_generator=sg, data_preprocessor=data_preprocessor)
batches=dl.load_batches(10,5,mgh_factors=[mg]) # used to populate normalization constants
y, mgh = data_preprocessor.convert_data()
x = data_preprocessor.one_hot_encode(bucket, sample_assignments, lower_dim=True)

In [8]:
losses = [
    'logspace_mse',
    'from_logspace_gil1',
    'from_logspace_gil1c',
    'from_logspace_l1',
    'from_logspace_l2',
    'from_logspace_gil2',
    # 'combined_gil1_ls_mse'
]

In [12]:
def evaluate_loss_fn(data_loader, config, Z, mg):
    net = Net(input_size=len(batches[0]['x'][0]), hidden_sizes=[100,100], use_gradient_values=True, device=device)
    t=Trainer(net, dataloader=dl, config=config)
    try:
        traced_losses_data = t.train(mgh_factors=[mg], traced_loss_fns=config['traced_loss_fns'])
        mess_hat = FastFactor.nn_to_FastFactor(idx, fastgm, data_preprocessor, net=net, device=device)
        return (mess_hat*mg).sum_all_entries() - Z, traced_losses_data
    except:
        # print the error
        print('didn\'t finish')
        mess_hat = FastFactor.nn_to_FastFactor(idx, fastgm, data_preprocessor, net=net, device=device)
        return (mess_hat*mg).sum_all_entries() - Z

In [10]:
results = []
scheduler_config = {
    'factor': 0.5,
    'patience': 4,
    'min_lr': 1e-8
}
test_config = {
    'loss_fn': 'logspace_mse',
    'traced_loss_fns': losses,
    'optimizer': 'adam',
    'learning_rate': 0.001,
    'device': device,
    'num_samples': 102400000,
    'batch_size': 10240,
    'num_epochs': 1,
    'set_size': 102400,
    'scheduler_config': scheduler_config
}

In [13]:
evaluate_loss_fn(dl, test_config, Z, mg)

Set 1/1000, Epoch 1/1, Loss: 0.04721791669726372
logspace_mse loss: 0.041318025439977646
from_logspace_gil1 loss: 1.1586179733276367
from_logspace_gil1c loss: 0.33326292037963867
from_logspace_l1 loss: 1.256537914276123
from_logspace_l2 loss: 3.4088077545166016
from_logspace_gil2 loss: -0.5582449436187744
Set 2/1000, Epoch 1/1, Loss: 0.036149222403764725
logspace_mse loss: 0.029850637540221214
from_logspace_gil1 loss: 0.16244935989379883
from_logspace_gil1c loss: 0.08672690391540527
from_logspace_l1 loss: 0.7617776393890381
from_logspace_l2 loss: 1.8925328254699707
from_logspace_gil2 loss: -1.4126520156860352
Set 3/1000, Epoch 1/1, Loss: 0.024510471150279045
logspace_mse loss: 0.018131379038095474
from_logspace_gil1 loss: 0.2895998954772949
from_logspace_gil1c loss: 0.28931331634521484
from_logspace_l1 loss: 0.8878507614135742
from_logspace_l2 loss: 2.145648956298828
from_logspace_gil2 loss: -1.0328943729400635
Set 4/1000, Epoch 1/1, Loss: 0.014143945649266243
logspace_mse loss: 0.0107

-0.48150634765625

In [11]:
traced_loss_data = {loss_fn: [] for loss_fn in losses}
num_iters = 2
for _ in range(num_iters):
    for loss_fn in losses[:2]:
        test_config['loss_fn'] = loss_fn
        print('testing ', 'loss_fn =', loss_fn)
        err, traced_loss_data_dict = evaluate_loss_fn(dl, test_config, Z, mg)
        traced_loss_data[loss_fn].append((err, traced_loss_data_dict))
        print('err: ', err)

testing  loss_fn = logspace_mse
Set 1/1000, Epoch 1/1, Loss: 0.01377920713275671
logspace_mse loss: 0.01058101560920477
from_logspace_gil1 loss: 2.1132760047912598
from_logspace_gil1c loss: 2.113260269165039
from_logspace_l1 loss: 2.091649055480957
from_logspace_l2 loss: 4.860503196716309
from_logspace_gil2 loss: 3.075993537902832
Set 2/1000, Epoch 1/1, Loss: 0.008040687069296837
logspace_mse loss: 0.0050306799821555614
from_logspace_gil1 loss: -1.5822221040725708
from_logspace_gil1c loss: -1.587712287902832
from_logspace_l1 loss: 1.7804179191589355
from_logspace_l2 loss: 4.315948963165283
from_logspace_gil2 loss: -5.718838691711426
Set 3/1000, Epoch 1/1, Loss: 0.0031097657047212124
logspace_mse loss: 0.0014668307267129421
from_logspace_gil1 loss: -5.845828056335449
from_logspace_gil1c loss: -7.220155715942383
from_logspace_l1 loss: 1.8091521263122559
from_logspace_l2 loss: 3.928269386291504
from_logspace_gil2 loss: -14.456125259399414
Set 4/1000, Epoch 1/1, Loss: 0.0014312028652057052

In [25]:
traced_loss_data[list(traced_loss_data.keys())[1]][1]

(-4.179962158203125,
 [[0,
   0.010774538852274418,
   2.300215721130371,
   2.300205707550049,
   1.8062753677368164,
   4.255607604980469,
   2.4284920692443848],
  [102400,
   0.005043818615376949,
   -1.1436858177185059,
   -1.1462901830673218,
   2.160165309906006,
   4.811283111572266,
   -3.7851481437683105],
  [204800,
   0.0013146218843758106,
   -6.186233043670654,
   -6.638864994049072,
   1.3737831115722656,
   3.100799083709717,
   -14.268470764160156],
  [307200,
   0.003669907571747899,
   -6.208678722381592,
   -6.209071159362793,
   2.056610107421875,
   4.706628799438477,
   -13.952960968017578],
  [409600,
   0.0064992755651474,
   -6.302439212799072,
   -6.302453994750977,
   1.6196131706237793,
   3.6649699211120605,
   -15.433109283447266],
  [512000,
   0.007973399013280869,
   -6.509742259979248,
   -6.509745121002197,
   2.1203927993774414,
   4.555887699127197,
   -16.103038787841797],
  [614400,
   0.008522715419530869,
   -6.368853569030762,
   -6.3688540458

In [ ]:
# sample_numbers = [row[0] for row in traced_loss_data]
# loss_data_dict = {losses[i]: [row[i+1] for row in traced_loss_data] for i in range(len(losses))}
# # change logspace_mse values to log_scale so they can be plotted together
# loss_data_dict['logspace_mse'] = [np.log(loss) for loss in loss_data_dict['logspace_mse']]

In [ ]:
# # Plot each loss function data
# for loss_name, loss_values in loss_data_dict.items():
#     plt.plot(sample_numbers, loss_values, label=loss_name)

# # Add labels, legend, and title
# plt.xlabel("Sample Numbers")
# plt.ylabel("Loss Values")
# plt.title("Losses over Samples")
# plt.legend()  # Automatically use the labels from the 'label' argument in plt.plot
# plt.grid(True)  # Optional: Add a grid for better readability

# # Show the plot
# plt.show()

In [ ]:
import pickle
with open('/home/cohenn1/NCE/notebooks/traced_loss_data.out2.pickle', 'rb') as f:
    loaded_dict = pickle.load(f)

In [2]:
loaded_dict['from_logspace_gil1'][0][1]

[[0,
  0.008466679602861404,
  1.1292507648468018,
  1.1291165351867676,
  2.892529010772705,
  6.268100261688232,
  -0.057646751403808594],
 [102400,
  0.003128011943772435,
  -3.0787129402160645,
  -3.1265926361083984,
  2.292372226715088,
  5.017903804779053,
  -8.117843627929688],
 [204800,
  0.0016160173108801246,
  -6.330998420715332,
  -6.369567394256592,
  2.335710048675537,
  5.205143451690674,
  -15.716791152954102],
 [307200,
  0.004383760038763285,
  -5.951046466827393,
  -5.951136589050293,
  2.3164992332458496,
  4.943568229675293,
  -14.105476379394531],
 [409600,
  0.006473221816122532,
  -6.13289737701416,
  -6.132901668548584,
  2.5199742317199707,
  5.52528715133667,
  -14.94947624206543],
 [512000,
  0.007476691156625748,
  -6.016822338104248,
  -6.016823768615723,
  2.1485342979431152,
  4.646579742431641,
  -13.944099426269531],
 [614400,
  0.00783312413841486,
  -6.147340297698975,
  -6.147341728210449,
  2.981307029724121,
  6.727038860321045,
  -15.631055831909